In [140]:
from Module.selenium_driver import webdriver_control
import sys
import re
import time
import pandas as pd

In [181]:
# 아티스트파인더 해당 페이지 아티스트 데이터 추출
def extract_artist_info(chrome):
    df_result = pd.DataFrame(columns=['artist_idx', 'artist_name'])
    # 해당 페이지의 아티스트 목록 추출
    artists = chrome.find_elements_by_css_selector('#pageList > div.list_atist12.d_artist_list > ul > li > div > div > dl > dt > a')
    # 아티스트 목록으로부터 artist_idx 추출
    for a in artists:
        artist_idx = re.findall('\d+', a.get_attribute('href'))[-1]
        df_result = df_result.append({'artist_idx':artist_idx, 'artist_name':a.text}, ignore_index=True)
    
    return df_result

In [187]:
# 먼저 cmd에서 크롬창 띄워줘야 자동화탭 안 뜸
# cd C:/Program Files/Google/Chrome/Application
# chrome.exe --remote-debugging-port=9222 --user-data-dir=C:/Chrome_debug_temp

# 드라이버 경로
DRIVER_PATH = 'D:/'

# 운영체제별 드라이버 선택
platform = sys.platform
# 윈도우일 경우
if platform == 'win32':
    DRIVER_PATH += 'chromedriver.exe'
# 리눅스일 경우
elif platform == 'linux':
    DRIVER_PATH += 'chromedriver_linux.exe'

# Chrome Driver init
chrome = webdriver_control.generate_chrome(driver_path=DRIVER_PATH)

In [ ]:
# 아티스트 이름 정렬 기준 선택
# '#conts > div.wrap_finder.atist > dl > dd > div:nth-child(1) > button:nth-child(16)' - 윗줄
# '#conts > div.wrap_finder.atist > dl > dd > div:nth-child(2) > button:nth-child(16)' - 아랫줄

# 아티스트 이름 정렬 기준 클릭
# chrome.find_element_by_css_selector(f'#conts > div.wrap_finder.atist > dl > dd > div:nth-child({1}) > button:nth-child({artist_sort_btn[1][6]})').click()

### 아이돌 아티스트 목록 수집
- 멜론 아티스트파인더의 아이돌 장르에 해당하는 아티스트 목록 수집 진행
- 가끔 아이돌이 아닌데 아이돌 장르로 분류된 아티스트가 있으니 주의 필요!
- EX) 김연자... 왜..?

In [188]:
# 아티스트 정렬 버튼을 위한 dict
# key = 윗줄(1) ,아랫줄 (2)
# value = 각 줄의 버튼
artist_sort_btn = {
    1 : [n for n in range(1, 17)],
    2 : [n for n in range(1, 27)],
}

# 아티스트파인더 창 띄우기
chrome.get('https://www.melon.com/artistplus/finder/index.htm')
time.sleep(1)
# 장르 > 아이돌 선택
chrome.find_element_by_xpath('//*[@id="GN2500"]').click()

In [193]:
df_idol = pd.DataFrame()

# 정렬 버튼대로 클릭 진행
for key, value in artist_sort_btn.items():
    
    for i in range(len(value)):
        # 아티스트 이름 정렬 기준 클릭
        chrome.find_element_by_css_selector(f'#conts > div.wrap_finder.atist > dl > dd > div:nth-child({key}) > button:nth-child({value[i]})').click()
        time.sleep(1)
        
        # 해당 페이지 아티스트 목록 추출
        df_result = extract_artist_info(chrome)
        # 다음 페이지로 넘기기
        pages = chrome.find_elements_by_css_selector('span.page_num > a')

        for i in range(len(pages)):
            pages[i].click()
            time.sleep(2)
            pages = chrome.find_elements_by_css_selector('span.page_num > a')

            df_result = pd.concat([df_result, extract_artist_info(chrome)], ignore_index=True)
        
        # 10페이지가 넘어가는지 아닌지 확인
        disable_btn = chrome.find_elements_by_xpath('//*[@id="pageObjNavgation"]/div/a[3]')
        
        # 버튼 있을 경우 (아예 데이터가 없어서 버튼도 없는 경우 존재)
        if disable_btn:
            disable_val = disable_btn[0].get_attribute('class')

            if 'disabled' in disable_val:
                print('10page 미만')
            else:
                print('10page 넘음')
                disable_btn[0].click()
                time.sleep(2)

                df_result = pd.concat([df_result, extract_artist_info(chrome)], ignore_index=True)

                # 다음 페이지로 넘기기
                pages = chrome.find_elements_by_css_selector('span.page_num > a')

                for i in range(len(pages)):
                    pages[i].click()
                    time.sleep(2)
                    pages = chrome.find_elements_by_css_selector('span.page_num > a')

                    df_result = pd.concat([df_result, extract_artist_info(chrome)], ignore_index=True)
                
        df_idol = pd.concat([df_idol, df_result], ignore_index=True)
        
df_idol.to_excel('idol_artist_list.xlsx', index=False, encoding='utf-8')

10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 넘음
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만
10page 미만


In [194]:
df_idol

,artist_idx,artist_name
0,28263,강승윤
1,205880,규현 (KYUHYUN)
2,828700,기현 (몬스타엑스)
3,428821,가인
4,482762,걸스데이
...,...,...
740,29674,Warren G
741,52607,XIA (준수)
742,2743578,X1 (엑스원)
743,895389,Young K


### 결과 저장

In [203]:
import pymysql
from config import config

In [205]:
# DB 연동
def connect_db_rds():
    conn = pymysql.connect(
        host = config.RDS_config['host']
        , user = config.RDS_config['user']
        , password = config.RDS_config['password']
        , database = config.RDS_config['database']
        , port = config.RDS_config['port']
        , cursorclass=pymysql.cursors.DictCursor
    )
    return conn

conn_rds = connect_db_rds()

In [200]:
for i in range(len(df_idol)):
    cursor = conn_rds.cursor()
    sql = config.query['insert_idol']
    val = (df_idol['artist_idx'][i], df_idol['artist_name'][i])
    cursor.execute(sql, val)
    conn_rds.commit()